In [1]:
import pandas as pd
pd.set_option('display.max_rows', None) # Print all rows

In [2]:
# FIX SSL: CERTIFICATE_VERIFY_FAILED with Python3
# https://stackoverflow.com/a/49174340
import ssl

ssl._create_default_https_context = ssl._create_unverified_context

In [3]:
url = 'https://www.juntadeandalucia.es/institutodeestadisticaycartografia/badea/stpivot/stpivot/Print?cube=013d884c-cfc5-4c21-9e78-e26225682afd&type=3&foto=si&ejecutaDesde=&codConsulta=38668&consTipoVisua=JP'
fields = ['Lugar de residencia', 'Medida', 'Valor']
df = pd.read_csv(url, delimiter=';', usecols=fields, decimal=',')

In [4]:
df[df['Lugar de residencia'] == 'Huelva']

,Lugar de residencia,Medida,Valor
0,Huelva,Población,513170
1,Huelva,Confirmados PDIA,29825
2,Huelva,Confirmados PDIA 14 días,1202
3,Huelva,Tasa PDIA 14 días,"234,230372001481"
4,Huelva,Confirmados PDIA 7 días,632
5,Huelva,Total Confirmados,30001
6,Huelva,Curados,26844
7,Huelva,Fallecidos,375


In [5]:
df['Valor'] = pd.to_numeric(df['Valor'],errors='coerce')

In [6]:
dfAnterior = pd.read_csv('covid_huelva.csv', index_col='Lugar de residencia')

In [7]:
# Confirmados PCR provincia Huelva
p = df[df['Lugar de residencia'] == 'Huelva'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]
print('Confirmados PCR en la provincia de Huelva: ', p)

Confirmados PCR en la provincia de Huelva:  29825.0
<ipython-input-7-31b732756b19>:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  p = df[df['Lugar de residencia'] == 'Huelva'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]


In [8]:
# Confirmados PCR Huelva capitalpd
c = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]
print('Confirmados PCR en Huelva capital: ', c)

Confirmados PCR en Huelva capital:  7801.0
<ipython-input-8-e81bfb40e710>:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  c = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]


In [9]:
c7 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 7 días']['Valor'].values[0]
c14 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 14 días']['Valor'].values[0]
p = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Población']['Valor'].values[0]

<ipython-input-9-f09dd2050cc1>:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  c7 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 7 días']['Valor'].values[0]
<ipython-input-9-f09dd2050cc1>:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  c14 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 14 días']['Valor'].values[0]
<ipython-input-9-f09dd2050cc1>:3: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  p = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Población']['Valor'].values[0]


In [10]:
print('Huelva (capital)\n')
#print('Confirmados PCR 7 días x 1,000 habitantes: ', f"{c7/(p/1000):.5f}")
print('Un positivo PCR cada', f"{p/c7:.0f}", 'personas en los últimos 7 días', '\n')
#print('Confirmados PCR 14 días x 1,000 habitantes: ', f"{c14/(p/1000):.5f}")
print('Un positivo PCR cada', f"{p/c14:.0f}", 'personas en los últimos 14 días')

Huelva (capital)

Un positivo PCR cada 795 personas en los últimos 7 días 

Un positivo PCR cada 385 personas en los últimos 14 días


In [11]:
filtro = ((df['Medida'] == 'Confirmados PDIA')| (df['Medida'] == 'Confirmados PDIA 7 días') | (df['Medida'] == 'Población' ) | (df['Medida'] == 'Confirmados PDIA 14 días')) & (df['Valor'] > 0)
tabla = pd.pivot_table(df.loc[filtro], values='Valor', index='Lugar de residencia', columns='Medida').dropna()

In [12]:
# Reorder columns
tabla = tabla[['Confirmados PDIA', 'Confirmados PDIA 7 días', 'Confirmados PDIA 14 días', 'Población']]

In [13]:
tabla['tasa7'] = tabla['Confirmados PDIA 7 días']/tabla['Población']*100000
tabla['tasa14'] = tabla['Confirmados PDIA 14 días']/tabla['Población']*100000

In [14]:
tabla['Diferencia PDIA'] = tabla['Confirmados PDIA'] - dfAnterior['Confirmados PDIA']

In [15]:
# Nuevos casos desde la última captura
tabla[tabla['Diferencia PDIA'] != 0].sort_values('Diferencia PDIA', ascending=False)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA
Lugar de residencia,,,,,,,
Huelva,29825.0,632.0,1202.0,513170.0,123.156069,234.230372,132.0
Condado-Campiña,9583.0,305.0,541.0,156231.0,195.223739,346.282108,65.0
Huelva-Costa,17264.0,275.0,573.0,289548.0,94.975617,197.894650,62.0
Huelva (capital),7801.0,181.0,374.0,143837.0,125.836885,260.016547,40.0
Palma del Condado (La),1091.0,43.0,53.0,10801.0,398.111286,490.695306,15.0
Almonte,1686.0,74.0,151.0,24507.0,301.954544,616.150488,14.0
Moguer,1181.0,56.0,76.0,21867.0,256.093657,347.555678,12.0
Bollullos Par del Condado,923.0,47.0,98.0,14387.0,326.683812,681.170501,10.0
Isla Cristina,2120.0,24.0,56.0,21393.0,112.186229,261.767868,8.0


In [16]:
if tabla[tabla['Diferencia PDIA'] != 0]['Diferencia PDIA'].sum() != 0:
  # Save to csv
  tabla.to_csv('covid_huelva.csv', encoding='utf-8', header=True, index=True)  

In [17]:
tabla.sort_values('tasa14', ascending=False)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA
Lugar de residencia,,,,,,,
Galaroza,41.0,6.0,14.0,1382.0,434.153401,1013.024602,0.0
Bollullos Par del Condado,923.0,47.0,98.0,14387.0,326.683812,681.170501,10.0
Almonte,1686.0,74.0,151.0,24507.0,301.954544,616.150488,14.0
Palma del Condado (La),1091.0,43.0,53.0,10801.0,398.111286,490.695306,15.0
Escacena del Campo,154.0,4.0,10.0,2287.0,174.901618,437.254045,0.0
Moguer,1181.0,56.0,76.0,21867.0,256.093657,347.555678,12.0
Condado-Campiña,9583.0,305.0,541.0,156231.0,195.223739,346.282108,65.0
Punta Umbría,765.0,29.0,52.0,15355.0,188.863562,338.651905,5.0
Niebla,208.0,7.0,13.0,4116.0,170.068027,315.840622,1.0


In [18]:
# Peso de la tasa de la última semana en relación con la tasa a 14 días 
# Mejor cuanto menor. Valor 1 significa que no había casos antes de 7 días.
tabla['7/14'] = tabla['tasa7'] / tabla['tasa14']
tabla.fillna(0).sort_values('7/14', ascending=True)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA,7/14
Lugar de residencia,,,,,,,,
Ayamonte,1298.0,2.0,12.0,21104.0,9.476876,56.861259,0.0,0.166667
Villablanca,185.0,1.0,5.0,2885.0,34.662045,173.310225,0.0,0.200000
Rociana del Condado,534.0,5.0,19.0,7939.0,62.980224,239.324852,0.0,0.263158
Palos de la Frontera,608.0,7.0,23.0,11742.0,59.615057,195.878045,3.0,0.304348
Bonares,261.0,2.0,6.0,6060.0,33.003300,99.009901,0.0,0.333333
Cartaya,1300.0,10.0,29.0,20083.0,49.793358,144.400737,6.0,0.344828
Beas,204.0,2.0,5.0,4341.0,46.072334,115.180834,0.0,0.400000
Escacena del Campo,154.0,4.0,10.0,2287.0,174.901618,437.254045,0.0,0.400000
Isla Cristina,2120.0,24.0,56.0,21393.0,112.186229,261.767868,8.0,0.428571
